# IBM Coursera Applied Data Science Capstone Project: 'The Battle of Neighborhoods'

# Report: Rise of the small bars – are there inner Sydney suburbs that are potentially untapped?

## Table of Contents

**Introduction**

* Background and business problem
  * Context
  * Business problem
  * Determination of suburbs of interest
  * Intended audience of report
  * Overview of methodology

* Data
  
  * Geocoding method
  * Data used


In [54]:
## Importing required libraries

import pandas as pd
import geopandas as gpd

from geopy.geocoders import Nominatim

import folium
from folium import plugins

import requests
import json
from pandas.io.json import json_normalize

# Introduction

## 1. Background and business problem

**Context**

For a particular market segment of bar patrons, inner Sydney has seen an emerging preference for small bars. While traditional Australian licensed venue types remain popular – pubs/hotels/bistros, sports and RSL clubs, nightclubs – the small bar is an emerging venue of choice for people looking for an intimate, ambient setting that is either quieter or meets an aesthetically distinct niche, compared to the larger licensed venues. The New South Wales government defines a small bar licence as follows: 'A small bar licence allows you to sell alcohol for consumption on the licensed premises, but does not allow gaming or take-away liquor. This liquor licence allows a maximum of 120 customers, over the age of 18, on the premises during authorised trading hours.' (https://www.service.nsw.gov.au/transaction/apply-small-bar-licence#:~:text=A%20small%20bar%20licence%20allows,premises%20during%20authorised%20trading%20hours).

The business opportunity arises particularly from the potential for conversion of unusual commercial properties into small bars. While many retailer types lose their physical presence in favour of online marketplaces, 'experience-based' venues – like small bars – remain a viable business option for the narrow lots in the commercial centres of inner Sydney suburbs.

**Business problem**

Small bars are a rising hospitality trend in Sydney's urban fringe outside the CBD, particularly in the inner west suburbs and in the areas just south of the CBD. 

There are opportunities for new small bar businesses in these suburbs but the selection of a potential location – with choices now opened up by options provided by smaller commercial lots of unusual types and special small bar licensing – is overwhelming. 

This report provides recommendations for the selection of small bar locations in Sydney's inner suburbs on the precipice of the CBD based on analysis of data from the Foursquare API and the Data.NSW open data portal.

**Determination of suburbs of interest**

Membership or partial membership within the City of Sydney Local Government Area, as defined on Wikipedia – https://en.wikipedia.org/wiki/City_of_Sydney – plus additional adjacent suburbs which share postcodes with these member suburbs (see Data for process of identifying the additional suburbs).

**Intended audience of report**

The scenario for the business problem is to determine the optimal locations for the establishment of new small bars in inner Sydney. The intended audience for this report includes hospitality and entertainment business owners, companies or investors looking to move, expand or establish into new locations in inner Sydney. 

Additional possible stakeholders would be local government councils, such as the City of Sydney and Inner West Councils, which have expressed strategic cultural and commercial interests in encouraging the night time economy in these areas, particularly for smaller capacity licensed venues.

**City of Sydney night-time economy page**

![](https://eugeneward.com.au/study/images/coursera_capstone/adsc_cpwk1_context_image1.jpg)


**Overview of methodology**

Below are the questions that direct the analysis. Each question is followed with a short overview of the approaches to the data in relation to the question.

* Which of these suburbs currently have the most small bars operating and can k-means cluster analysis for suburb similarity shed any insight on which suburbs might otherwise have similar sets of activities and venues but a lack of small bars?
  * Datasets used: Foursquare API data; Data.NSW licensed premises list.

* Are there suburbs which scored highly on relevant liveability indicators in the Liveable Sydney Study dataset which: have small bars operating? Do not have any small bars operating?
  * Datasets used: Domain Liveable Sydney Study 2019

* Are there any patterns regarding surrounding venues for the localised radius of a small bar (or cluster of small bars) and can these be explored and potentially revealed via k-means clustering?
  * Datasets used: Foursquare API data (with API calls set to smaller radius); Data.NSW licensed premises list for precise geocoding of existing small bars.



## 2. Data


**Geocoding method**: 

For suburbs – Nominatim from the geopy client. Returned geocoded point data is at suburb-level.


In [6]:
## Example of suburb feature geocoding method

suburb = 'Newtown, Australia'
geolocator = Nominatim(user_agent="au_explorer")
location = geolocator.geocode(suburb)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(suburb, latitude, longitude))

The geograpical coordinates of Newtown, Australia are -33.8961132, 151.1801893.


**Data used – each dataset is bolded and then described with examples of use:**

**Foursquare API** (https://developer.foursquare.com/)
  * Method: Query for nearby venues for each suburb using centroid coordinates (as geocoded by Nominatim) as centre of radius. Collect results and examine density of bars and licensed venues for the suburbs of interest.


In [57]:
## Example of Foursquare API data use – query for one suburb of interest

# Define Foursquare credentials and version

CLIENT_ID = 'KRND1MPPQRN3RU32M00CFTAH32XMHVJBX4F2VNRQEEA0PVQQ' # unique Foursquare Client ID
CLIENT_SECRET = 'G2FQJOONTKX2MRFYCDJU4APRFDTXTBJO5XF4M5SEI1F2NV5X' # unique Foursquare Client Secret
VERSION = '20200802' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

# Define parameters for API calls

LIMIT = 200 # Limit on number of venues returned
radius = 500 # in metres

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

Your credentials:
CLIENT_ID: KRND1MPPQRN3RU32M00CFTAH32XMHVJBX4F2VNRQEEA0PVQQ
CLIENT_SECRET: G2FQJOONTKX2MRFYCDJU4APRFDTXTBJO5XF4M5SEI1F2NV5X


,name,categories,lat,lng
0,Black Star Pastry,Bakery,-33.897172,151.178623
1,Gelato Messina,Ice Cream Shop,-33.896060,151.181170
2,Dendy Cinemas,Indie Movie Theater,-33.896094,151.180544
3,Gelato Blue,Ice Cream Shop,-33.897192,151.179316
4,Continental Deli Bar Bistro,Deli / Bodega,-33.896342,151.178397
5,Delhi 'O' Delhi,Indian Restaurant,-33.897142,151.180542
6,Black Sheep,Cocktail Bar,-33.895758,151.181115
7,Thai Pothong,Thai Restaurant,-33.896703,151.179702
8,Mary's,Bar,-33.896124,151.179791
9,The Italian Bowl,Italian Restaurant,-33.895948,151.180877


In [56]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

53 venues were returned by Foursquare.


**Data.NSW – Liquor Licenced Premises List June 2020** (https://data.nsw.gov.au/data/dataset/liquor-licence-premises-list/resource/bda22b68-8e4f-4028-b725-52dc0912a626)
  * Additional dataset to enhance the venue data for suburbs (to supplement the venue data retrieved via the Foursquare API) and to provide precise information for existing small bar licences and their locations.


In [4]:
## Example of Data.NSW Licensed Premises List data use

## Loading in the data
nswprem = pd.read_csv('https://data.nsw.gov.au/data/dataset/297dfb0e-3b94-433b-be64-3bb0905e28a4/resource/bda22b68-8e4f-4028-b725-52dc0912a626/download/ebethanypremises-list-as-at-june-2020.csv', encoding='ISO-8859-1')
print(nswprem.head())

     Licence No.           Licence Type   Status  Start Date  \
0  LIQC300200019  Liquor - club licence  Current  29/05/1981   
1  LIQC300200035  Liquor - club licence  Current  21/08/1981   
2  LIQC300200078  Liquor - club licence  Current   3/12/1981   
3  LIQC300200086  Liquor - club licence  Current  23/12/1981   
4  LIQC300200094  Liquor - club licence  Current  15/03/1982   

                               Licence Name           Address        Suburb  \
0                 Charlestown Golf Club Ltd     1A Barker Ave  HILLSBOROUGH   
1                   Barooga Sports Club Ltd  12 Burkinshaw St       BAROOGA   
2  Maclean-Lower Clarence Services Club Ltd    36-38 River St       MACLEAN   
3                         Sporties Tuncurry          Beach St      TUNCURRY   
4  Canley Heights R S L & Sporting Club Ltd   26 Humphries Rd       WAKELEY   

   Postcode   Latitude   Longitude  ... After 3am After 5am  \
0    2290.0 -32.963633  151.669816  ...       Yes       Yes   
1    3644.0 -3

In [16]:
## IDA / context example – number of small bar licences in entire state
sum(nswprem['Licence Type']=="Liquor - small bar licence")

150

In [8]:
## Exploring current liquor licences for a particular suburb

nswprem_newtown = nswprem.loc[nswprem.Suburb=='NEWTOWN']
nswprem_newtown

,Licence No.,Licence Type,Status,Start Date,Licence Name,Address,Suburb,Postcode,Latitude,Longitude,...,After 3am,After 5am,SA2,Band,GMEs,PMPs,Auth Limit,Unfld Quota,GMT,EGMs
1316,LIQH400100345,Liquor - hotel licence,Current,3/11/1958,Bank Hotel,324 King St,NEWTOWN,2042.0,-33.897519,151.179371,...,Yes,Yes,Newtown - Camperdown - Darlington,1.0,15.0,0.0,15.0,0.0,15.0,15.0
1342,LIQH400100655,Liquor - hotel licence,Current,15/02/1960,Botany View Hotel,597 King St,NEWTOWN,2042.0,-33.905846,151.180727,...,No,No,Newtown - Camperdown - Darlington,1.0,12.0,0.0,12.0,0.0,12.0,12.0
1377,LIQH400101074,Liquor - hotel licence,Current,1/08/1956,Carlisle Castle Hotel,Albermarle & Regent Sts,NEWTOWN,2042.0,-33.895337,151.176451,...,No,No,Newtown - Camperdown - Darlington,1.0,10.0,0.0,10.0,0.0,10.0,10.0
1414,LIQH400101503,Liquor - hotel licence,Current,28/05/1956,Court House Hotel,202-204 Australia St,NEWTOWN,2042.0,-33.896080,151.178371,...,No,No,Newtown - Camperdown - Darlington,1.0,14.0,0.0,14.0,0.0,14.0,13.0
1607,LIQH400103743,Liquor - hotel licence,Current,29/09/1958,Marlborough Hotel,145 King St,NEWTOWN,2042.0,-33.893278,151.183189,...,Yes,Yes,Newtown - Camperdown - Darlington,1.0,15.0,15.0,30.0,0.0,30.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16277,LIQS220000106,Liquor - small bar licence,Current,4/06/2018,Blue Fig Cafe Bar,Shop 1 112-116 Enmore Rd,NEWTOWN,2042.0,-33.898982,151.174574,...,No,No,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
16285,LIQS220000115,Liquor - small bar licence,Current,22/08/2018,Tandem Bar,127 King St,NEWTOWN,2042.0,-33.892999,151.183815,...,No,No,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
16307,LIQS220000137,Liquor - small bar licence,Current,30/01/2019,2T Newtown Bar,169 King St,NEWTOWN,2042.0,-33.893872,151.182696,...,No,No,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
17223,LIQW880010245,Liquor - producer wholesaler licence,Current,13/09/2010,Sparrow and Vine Wholesale,72 Hordern St,NEWTOWN,2042.0,-33.893353,151.180694,...,No,No,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
catdf = pd.DataFrame(nswprem_newtown['Licence Type'], dtype="category")
catdf.describe()

,Licence Type
count,132
unique,5
top,Liquor - on-premises licence
freq,96


In [47]:
## Mapping the licenses to a suburb geography with aesthetic for license type - to explore any trends

colordict = {'Liquor - hotel licence': 'purple', 'Liquor - on-premises licence': 'lightblue', 'Liquor - packaged liquor licence': 'orange', 'Liquor - small bar licence': 'red', 'Liquor - producer wholesaler licence': 'beige'}

bmap_eg = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=15)

for lat, lon, poi, lictype, licyear in zip(nswprem_newtown['Latitude'], nswprem_newtown['Longitude'], nswprem_newtown['Licence Name'], nswprem_newtown['Licence Type'], nswprem_newtown['Start Date']):
    label = '{}, {}'.format(poi, lictype)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        popup = label,
        color = lictype,
        fill_color = colordict[lictype],
        fill = True,
        fill_opacity=0.7
        ).add_to(bmap_eg)

bmap_eg

Output of the Folium code above as static image:

**Liquor Licences in Newtown (June 2020)**

![](https://eugeneward.com.au/study/images/coursera_capstone/adsc_cpwk1_map1.jpg)

**Australian Local Government Area to Included Postcode Mappings** – prepared by Jeremy Epstein (http://greenash.net.au/) from 2011 Australian Bureau of Statistics data (https://greenash.net.au/thoughts/2014/07/australian-lga-to-postcode-mappings-with-postgis-and-intersects/)
  
**Australia Post postcode lookup** (https://auspost.com.au/postcode)  
  
  * These two sources were used for determining additional suburbs which are adjacent to City of Sydney LGA suburbs.


In [58]:
## This is the file that has been generated from the Wikipedia page, 
## greenash/ABS data and Australia Post postcode service:

thesuburbs = pd.read_csv('https://eugeneward.com.au/study/demodata/Sydney_C_LGA_Postcodes_Suburbs.csv')
thesuburbs

,Postcode,Suburb
0,2000,Barangaroo
1,2000,Dawes Point
2,2000,Haymarket
3,2000,Millers Point
4,2000,Sydney
5,2000,Sydney South
6,2000,The Rocks
7,2007,Broadway
8,2007,Ultimo
9,2008,Chippendale


Suburb membership, partial membership or shared postcodes with suburbs with membership in the boundaries of the City of Sydney Local Government Area is the criteria for suburbs of interest. This table is the contextual data that describes the suburbs of interest.

**Domain Group, Tract and Deloitte Access Economics – Liveable Sydney Study 2019** (https://www.domain.com.au/liveable-sydney/sydneys-most-liveable-suburbs-2019/sydneys-569-suburbs-ranked-for-liveability-2019-903130/)
  * This geojson dataset is an analysis of 19 'liveability indicators' for all of the suburbs in Sydney – it also then uses bucket evaluation to rank all suburbs for liveability.

In [59]:
## Preview of Domain Liveable Sydney Study data

import geopandas as gpd

domainlive = gpd.read_file('https://static.domain.com.au/content/fe-static/%40domain-group/fe-co-content-data-map/data/Liveability_GEOJSON_NSW.json')
print(domainlive.head())

   airport_noise_score  bucket_rank  bus_score  cafe_tot_score  \
0                    1            4          2               5   
1                    1            4          4               5   
2                    1            5          2               5   
3                    1           12          3               5   
4                    1            8          4               4   

   coast_beach_score  congestion_score  crime_score  culture_score  \
0                  4                 1            1              5   
1                  4                 1            1              5   
2                  4                 1            2              5   
3                  4                 1            1              5   
4                  4                 1            1              5   

   educ_prim_score  educ_sec_score  ...  state        suburb  topo_var_score  \
0                5               5  ...    NSW       Newtown               3   
1                3    

 In particular, suburb scores for **public transport, walkability and culture** will help to inform the recommendations around suburbs for new small bars. This dataset can also be useful because it provides geometries for all the suburbs (which are not actually administrative areas and therefore their boundary shapes are not featured on open data resources).